In [9]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
import plotly.express as px

import bmll2 as b2
from bmll2 import reference, Security, NormalisedSecurity, SparkHelper, get_market_data, get_market_data_range, VenueMarketError, save_spark_dataframe, load_spark_dataframe

pd.options.display.max_columns = None

In [8]:
def get_data(ticker, starting_date, ending_date):

    #if isinstance(startingdate, str):
    #    date = pd.to_datetime(date).date()
    
    try:
        trade_plus = get_market_data_range('XJSE', start_date = starting_date, end_date = ending_date,
                                   table_name = 'trades-plus', df_engine = 'polars', ticker = ticker,
                                   columns = ['Classification', 'MIC', 'Ticker', 'ListingId', 'TradeDate',
                                              'LocalTradeTimestamp', 'ExchangeSequenceNo', 'AggressorSide',
                                              'Price', 'Size', 'PreTradeMid1ms', 'PostTradeMid1ms'])
    except VenueMarketError:
        return None
        
    trade_plus = trade_plus.to_pandas()
    trade_plus = trade_plus[(trade_plus['Classification'] == 'LIT_CONTINUOUS')]
    
    if trade_plus is None or trade_plus.empty:
        return None
        
    trade_plus = trade_plus[['MIC', 'Ticker', 'ListingId', 'TradeDate', 'LocalTradeTimestamp', 'ExchangeSequenceNo',
                             'AggressorSide', 'Price', 'Size', 'PreTradeMid1ms', 'PostTradeMid1ms']]
    trade_plus['AggressorSide'] = trade_plus['AggressorSide'].map({1 : 1, 2 : -1, 0 : 0})
    trade_plus = trade_plus.rename(columns = {'AggressorSide' : 'Trade Sign', 'PreTradeMid1ms' : 'Mid-price before',
                                              'PostTradeMid1ms' : 'Mid-price after(immediate)', 'Size' : 'Volume',
                                              'LocalTradeTimestamp' : 'DateTime', 'TradeDate' : 'Date'})
    midprice_after_delayed = trade_plus['Mid-price before'].shift(-1)
    trade_plus.insert(loc = 11, column = 'Mid-price after(delayed)', value = midprice_after_delayed)
    
    l1 = get_market_data_range('XJSE', start_date = starting_date, end_date = ending_date, ticker = ticker,
                               table_name = 'l1', df_engine = 'polars')
    l1 = l1.to_pandas()
    l1 = l1[l1['MarketState'] == 'CONTINUOUS_TRADING']
    midprice = (l1['AskPrice1'] + l1['BidPrice1']) / 2
    l1.insert(loc = 5, column = 'Mid-price', value = midprice)
    l1 = l1.rename(columns = {'TradeDate' : 'Date'})
    l1 = l1.sort_values(by = ['Date', 'ExchangeSequenceNo'])
    daily_vol = (l1.groupby('Date', as_index = False)['Mid-price'].apply(lambda x: (x.max() - x.min()) / x.iloc[0])).rename(columns = {'Mid-price' : 'Daily Volatility'})
    daily_vol_alt = (l1.groupby('Date', as_index = False)['Mid-price'].apply(lambda x: np.log(x.max()) - np.log(x.min()))).rename(columns = {'Mid-price' : 'Daily Volatility(alt)'})
    trade_plus = pd.merge(trade_plus, daily_vol, on = 'Date', how = 'left')
    trade_plus = pd.merge(trade_plus, daily_vol_alt, on = 'Date', how = 'left')

    col = 'Daily Volatility'
    cols = list(trade_plus.columns)
    cols.insert(6, cols.pop(cols.index(col)))
    
    #col = 'Daily Volatility(alt)'
    #cols = list(trade_plus.columns)
    #cols.insert(7, cols.pop(cols.index(col)))
    
    trade_plus = trade_plus[cols]
    daily_volume = (trade_plus.groupby('Date', as_index = False)['Volume'].sum().rename(columns = {'Volume' : 'Daily Volume'}))
    trade_plus = pd.merge(trade_plus, daily_volume, on = 'Date', how = 'left')
    col = 'Daily Volume'
    cols = list(trade_plus.columns)
    cols.insert(6, cols.pop(cols.index(col)))
    trade_plus = trade_plus[cols]
    #trade_plus.insert(loc = 6, column = 'Daily Volume', value = trade_plus['Volume'].sum())
    
    trade_plus.sort_values(['DateTime', 'ExchangeSequenceNo'])
    return trade_plus
    

In [11]:
b2.get_file('other/JSE_listed_securities.xlsx')
JSE_listed_securities = pd.read_excel('JSE_listed_securities.xlsx')

JSE_listed_securities = JSE_listed_securities.drop(columns = ['Value', 'Change', 'Unnamed: 4', 'High', 'Low'])
JSE_listed_securities = JSE_listed_securities.fillna(0)

In [12]:
def clean(x):
    if(isinstance(x, (int, float))):
        ans = x
    elif(x[-1:] == 'M'):
        ans = float(x[:-1]) * 1000000
    elif(x[-1:] == 'K'):
        ans = float(x[:-1]) * 1000
    else:
        ans = 0
    return ans

In [13]:
JSE_listed_securities['Volume'] = JSE_listed_securities['Volume'].apply(clean)
JSE_listed_securities = JSE_listed_securities.sort_values('Volume', ascending = False)

In [7]:
top100_tickers = []
i = 0
while len(top100_tickers) < 100:
    test = get_data(JSE_listed_securities['Symbol'].iloc[i], '2025-12-01', '2025-12-07')
    if test is None:
        print(JSE_listed_securities['Symbol'].iloc[i])
    else:
        top100_tickers.append(JSE_listed_securities['Symbol'].iloc[i])
    i += 1

NHM
ECS
RMI
AHB
TCP
CCO
CTA
AMS


In [7]:
start_dates = pd.date_range(start = '2023-01-01', end = '2025-12-31', freq = 'MS', inclusive = 'both').date
end_dates   = pd.date_range(start = '2023-01-01', end = '2025-12-31', freq = 'M', inclusive = 'both').date

start_dates = start_dates[::3]
end_dates   = end_dates[2::3]

In [18]:
%%time
for ticker in top100_tickers:
    print(ticker)
    stock = []
    for i in range(len(start_dates)):
        data = get_data(ticker, start_dates[i], end_dates[i])
        if data is not None:
            stock.append(data)
    
    stock_data = pd.concat(stock, ignore_index = True)
    stock_data.to_csv(f'{ticker}.csv', index = False)
    b2.put_file(f'{ticker}.csv', 'top_100(Volume)')

# takes about 5 hours to run

LAB
PPH
FSR
OMU
WHL
SSW
FTB
ORN
PAN
GLN
PIK
RDF
OUT
SOL
GRT
MTN
IMP
LHC
SAP
SLM
SAC
TRU
OPA
SBK
ABG
VKE
NTC
CPR
SSU
DRD
KP2
HAR
MTM
KAP
DCP
N91
DSY
GFI
APN
NPH
DIB
MDI
NPN
AEL
SHP
GND
CCD
QLT
CML
PPC
VAL
INL
INP
MRP
JBL
BLU
NED
S32
TFG
VOD
NRP
BYI
NY1
REM
KST
MNP
BVT
FFB
ACL
ANG
BTI
SRE
CLS
ITE
PRX
ARI
RBO
SPG
WBC
LTE
SSS
AGL
BID
PPE
AFT
TKG
EQU
YRK
MSP
TGA
AEG
OMN
BTN
AFE
AVI
EXX
RNI
BHG
BOX
ANH
CPU times: user 7h 8min 41s, sys: 50min 38s, total: 7h 59min 19s
Wall time: 5h 8min 33s


In [20]:
# I need to do this because the ticker for VAL was AMS before 2025-05-28
%%time
start_dates = pd.date_range(start = '2023-01-01', end = '2025-04-30', freq = 'MS', inclusive = 'both').date
end_dates   = pd.date_range(start = '2023-01-01', end = '2025-04-30', freq = 'M', inclusive = 'both').date

VAL = []
for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('AMS', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

start_dates = pd.date_range(start = '2025-05-01', end = '2025-05-27', freq = 'B', inclusive = 'both').date
end_dates   = pd.date_range(start = '2025-05-01', end = '2025-05-27', freq = 'B', inclusive = 'both').date

for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('AMS', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

start_dates = pd.date_range(start = '2025-05-27', end = '2025-05-31', freq = 'B', inclusive = 'both').date
end_dates   = pd.date_range(start = '2025-05-27', end = '2025-05-31', freq = 'B', inclusive = 'both').date

for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('VAL', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

start_dates = pd.date_range(start = '2025-06-01', end = '2025-12-31', freq = 'MS', inclusive = 'both').date
end_dates   = pd.date_range(start = '2025-06-01', end = '2025-12-31', freq = 'M', inclusive = 'both').date

for i in range(len(start_dates)):
    print(start_dates[i], end_dates[i])
    data = get_data('VAL', start_dates[i], end_dates[i])
    if data is not None:
        VAL.append(data)

VAL_data = pd.concat(VAL, ignore_index = True)
VAL_data['Ticker'] = 'VAL'
VAL_data.to_csv('VAL.csv', index = False)
b2.put_file('VAL.csv', 'top_100(Volume)')

2023-01-01 2023-01-31
2023-02-01 2023-02-28
2023-03-01 2023-03-31
2023-04-01 2023-04-30
2023-05-01 2023-05-31
2023-06-01 2023-06-30
2023-07-01 2023-07-31
2023-08-01 2023-08-31
2023-09-01 2023-09-30
2023-10-01 2023-10-31
2023-11-01 2023-11-30
2023-12-01 2023-12-31
2024-01-01 2024-01-31
2024-02-01 2024-02-29
2024-03-01 2024-03-31
2024-04-01 2024-04-30
2024-05-01 2024-05-31
2024-06-01 2024-06-30
2024-07-01 2024-07-31
2024-08-01 2024-08-31
2024-09-01 2024-09-30
2024-10-01 2024-10-31
2024-11-01 2024-11-30
2024-12-01 2024-12-31
2025-01-01 2025-01-31
2025-02-01 2025-02-28
2025-03-01 2025-03-31
2025-04-01 2025-04-30
2025-05-01 2025-05-01
2025-05-02 2025-05-02
2025-05-05 2025-05-05
2025-05-06 2025-05-06
2025-05-07 2025-05-07
2025-05-08 2025-05-08
2025-05-09 2025-05-09
2025-05-12 2025-05-12
2025-05-13 2025-05-13
2025-05-14 2025-05-14
2025-05-15 2025-05-15
2025-05-16 2025-05-16
2025-05-19 2025-05-19
2025-05-20 2025-05-20
2025-05-21 2025-05-21
2025-05-22 2025-05-22
2025-05-23 2025-05-23
2025-05-26

In [10]:
top100_tickers = pd.DataFrame(top100_tickers)
top100_tickers.to_csv('top100_tickers.csv', index = False)
b2.put_file('top100_tickers.csv')